# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f1713ef31f0>
├── 'a' --> tensor([[-0.6658, -0.4001,  1.4582],
│                   [-0.7380, -1.4507,  1.1571]])
└── 'x' --> <FastTreeValue 0x7f173c7a7520>
    └── 'c' --> tensor([[ 0.2682,  0.4197, -0.5210,  0.0046],
                        [ 0.3191, -0.7034,  0.5033, -0.6982],
                        [-0.0072,  0.2289, -1.4102,  1.6873]])

In [4]:
t.a

tensor([[-0.6658, -0.4001,  1.4582],
        [-0.7380, -1.4507,  1.1571]])

In [5]:
%timeit t.a

78.5 ns ± 0.188 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f1713ef31f0>
├── 'a' --> tensor([[ 0.0687,  0.9521, -0.1747],
│                   [ 0.7854, -1.0446, -1.3658]])
└── 'x' --> <FastTreeValue 0x7f173c7a7520>
    └── 'c' --> tensor([[ 0.2682,  0.4197, -0.5210,  0.0046],
                        [ 0.3191, -0.7034,  0.5033, -0.6982],
                        [-0.0072,  0.2289, -1.4102,  1.6873]])

In [7]:
%timeit t.a = new_value

79.3 ns ± 0.363 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.6658, -0.4001,  1.4582],
               [-0.7380, -1.4507,  1.1571]]),
    x: Batch(
           c: tensor([[ 0.2682,  0.4197, -0.5210,  0.0046],
                      [ 0.3191, -0.7034,  0.5033, -0.6982],
                      [-0.0072,  0.2289, -1.4102,  1.6873]]),
       ),
)

In [10]:
b.a

tensor([[-0.6658, -0.4001,  1.4582],
        [-0.7380, -1.4507,  1.1571]])

In [11]:
%timeit b.a

70.2 ns ± 0.406 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.8217,  2.9410,  0.8349],
               [ 0.1392, -0.2347, -0.8072]]),
    x: Batch(
           c: tensor([[ 0.2682,  0.4197, -0.5210,  0.0046],
                      [ 0.3191, -0.7034,  0.5033, -0.6982],
                      [-0.0072,  0.2289, -1.4102,  1.6873]]),
       ),
)

In [13]:
%timeit b.a = new_value

592 ns ± 2.44 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

969 ns ± 3.47 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13 µs ± 164 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

167 µs ± 1.5 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

165 µs ± 885 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f1693f42c40>
├── 'a' --> tensor([[[-0.6658, -0.4001,  1.4582],
│                    [-0.7380, -1.4507,  1.1571]],
│           
│                   [[-0.6658, -0.4001,  1.4582],
│                    [-0.7380, -1.4507,  1.1571]],
│           
│                   [[-0.6658, -0.4001,  1.4582],
│                    [-0.7380, -1.4507,  1.1571]],
│           
│                   [[-0.6658, -0.4001,  1.4582],
│                    [-0.7380, -1.4507,  1.1571]],
│           
│                   [[-0.6658, -0.4001,  1.4582],
│                    [-0.7380, -1.4507,  1.1571]],
│           
│                   [[-0.6658, -0.4001,  1.4582],
│                    [-0.7380, -1.4507,  1.1571]],
│           
│                   [[-0.6658, -0.4001,  1.4582],
│                    [-0.7380, -1.4507,  1.1571]],
│           
│                   [[-0.6658, -0.4001,  1.4582],
│                    [-0.7380, -1.4507,  1.1571]]])
└── 'x' --> <FastTreeValue 0x7f1693f426a0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

37.3 µs ± 307 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f1693f42d30>
├── 'a' --> tensor([[-0.6658, -0.4001,  1.4582],
│                   [-0.7380, -1.4507,  1.1571],
│                   [-0.6658, -0.4001,  1.4582],
│                   [-0.7380, -1.4507,  1.1571],
│                   [-0.6658, -0.4001,  1.4582],
│                   [-0.7380, -1.4507,  1.1571],
│                   [-0.6658, -0.4001,  1.4582],
│                   [-0.7380, -1.4507,  1.1571],
│                   [-0.6658, -0.4001,  1.4582],
│                   [-0.7380, -1.4507,  1.1571],
│                   [-0.6658, -0.4001,  1.4582],
│                   [-0.7380, -1.4507,  1.1571],
│                   [-0.6658, -0.4001,  1.4582],
│                   [-0.7380, -1.4507,  1.1571],
│                   [-0.6658, -0.4001,  1.4582],
│                   [-0.7380, -1.4507,  1.1571]])
└── 'x' --> <FastTreeValue 0x7f1693f42520>
    └── 'c' --> tensor([[ 0.2682,  0.4197, -0.5210,  0.0046],
                        [ 0.3191, -0.7034,  0.5033, -0.6982],
                 

In [23]:
%timeit t_cat(trees)

38.3 µs ± 796 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

63.7 µs ± 315 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.6658, -0.4001,  1.4582],
                [-0.7380, -1.4507,  1.1571]],
       
               [[-0.6658, -0.4001,  1.4582],
                [-0.7380, -1.4507,  1.1571]],
       
               [[-0.6658, -0.4001,  1.4582],
                [-0.7380, -1.4507,  1.1571]],
       
               [[-0.6658, -0.4001,  1.4582],
                [-0.7380, -1.4507,  1.1571]],
       
               [[-0.6658, -0.4001,  1.4582],
                [-0.7380, -1.4507,  1.1571]],
       
               [[-0.6658, -0.4001,  1.4582],
                [-0.7380, -1.4507,  1.1571]],
       
               [[-0.6658, -0.4001,  1.4582],
                [-0.7380, -1.4507,  1.1571]],
       
               [[-0.6658, -0.4001,  1.4582],
                [-0.7380, -1.4507,  1.1571]]]),
    x: Batch(
           c: tensor([[[ 0.2682,  0.4197, -0.5210,  0.0046],
                       [ 0.3191, -0.7034,  0.5033, -0.6982],
                       [-0.0072,  0.2289, -1.4102,  1.6873]],
         

In [26]:
%timeit Batch.stack(batches)

94.4 µs ± 3.19 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.6658, -0.4001,  1.4582],
               [-0.7380, -1.4507,  1.1571],
               [-0.6658, -0.4001,  1.4582],
               [-0.7380, -1.4507,  1.1571],
               [-0.6658, -0.4001,  1.4582],
               [-0.7380, -1.4507,  1.1571],
               [-0.6658, -0.4001,  1.4582],
               [-0.7380, -1.4507,  1.1571],
               [-0.6658, -0.4001,  1.4582],
               [-0.7380, -1.4507,  1.1571],
               [-0.6658, -0.4001,  1.4582],
               [-0.7380, -1.4507,  1.1571],
               [-0.6658, -0.4001,  1.4582],
               [-0.7380, -1.4507,  1.1571],
               [-0.6658, -0.4001,  1.4582],
               [-0.7380, -1.4507,  1.1571]]),
    x: Batch(
           c: tensor([[ 0.2682,  0.4197, -0.5210,  0.0046],
                      [ 0.3191, -0.7034,  0.5033, -0.6982],
                      [-0.0072,  0.2289, -1.4102,  1.6873],
                      [ 0.2682,  0.4197, -0.5210,  0.0046],
                      [ 0.3191, -

In [28]:
%timeit Batch.cat(batches)

168 µs ± 1.33 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

374 µs ± 10.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
